In [ ]:
import os
import re
from pathlib import Path
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nptdms import TdmsFile


# ------------------ Hjælpefunktioner ------------------

def extract_patient_id_from_path(p: Path) -> int | None:
    """Forsøger at finde et patient-ID i sti/filnavn (Patient 5, Patient_5, etc.)."""
    parts = list(p.parts) + [p.stem]
    for part in parts:
        m = re.search(r'Patient[ _-]?(\d+)', part, flags=re.IGNORECASE)
        if m:
            try:
                return int(m.group(1))
            except ValueError:
                pass
    return None


def detect_time_columns(df: pd.DataFrame):
    """
    Gæt kolonner for start/slut (dansk/engelsk varianter).
    Returnerer (start_col, end_col) eller (col, None) hvis kun start.
    """
    cols = {c.lower(): c for c in df.columns.astype(str)}
    # kandidater
    start_keys = ["start", "starttid", "start time", "starttime", "anfaldsstart", "onset", "begin"]
    end_keys   = ["slut", "sluttid", "end", "end time", "endtime", "offset", "stop"]

    def pick(keys):
        for k in cols:
            if any(key in k for key in keys):
                return cols[k]
        return None

    c_start = pick(start_keys)
    c_end   = pick(end_keys)
    if c_start is None:
        # Hvis ingen oplagt start-kolonne, prøv første datetime-agtige kolonne
        for c in df.columns:
            if np.issubdtype(df[c].dtype, np.datetime64):
                c_start = c
                break
    return c_start, c_end

import pandas as pd
import numpy as np
from pathlib import Path

def _coerce_datetime(series: pd.Series) -> pd.Series:
    # Numerisk? -> Excel-serial (origin 1899-12-30)
    if np.issubdtype(series.dtype, np.number):
        return pd.to_datetime(series, unit="d", origin="1899-12-30", errors="coerce")
    # Allerede datetime?
    if np.issubdtype(series.dtype, np.datetime64):
        return pd.to_datetime(series, errors="coerce")
    # Tekst: prøv dayfirst=True
    return pd.to_datetime(series.astype(str).str.strip(), errors="coerce", dayfirst=True)

# def load_annotations_from_excel_kvj(xls_path, start_row=8):
    """
    Læs tider fra Excel:
      - Start: kolonne E (fra række 8), fallback D
      - Slut:  kolonne F (hvis tilgængelig)
    Returnerer liste af dicts: [{"start": dt, "end": dt|None, "row": excel_row}, ...]
    """
    import pandas as pd, numpy as np
    from pathlib import Path
    xls_path = Path(xls_path)
    engine = "xlrd" if xls_path.suffix.lower() == ".xls" else None
    df = pd.read_excel(xls_path, header=None, engine=engine)

    r0 = max(0, start_row - 1)
    sub = df.iloc[r0:].copy()

    def col_idx(letter): return ord(letter.upper()) - ord('A')
    idx_D, idx_E, idx_F = col_idx('D'), col_idx('E'), col_idx('F')

    def coerce(s):
        # tal -> Excel-serial; datetime -> behold; tekst -> parse (dayfirst)
        if np.issubdtype(s.dtype, np.number):
            return pd.to_datetime(s, unit="d", origin="1899-12-30", errors="coerce")
        if np.issubdtype(s.dtype, np.datetime64):
            return pd.to_datetime(s, errors="coerce")
        return pd.to_datetime(s.astype(str).str.strip(), errors="coerce", dayfirst=True)

    in_cols = set(sub.columns)
    if idx_E in in_cols:
        s_start = coerce(sub[idx_E])
    elif idx_D in in_cols:
        s_start = coerce(sub[idx_D])
    else:
        raise ValueError("Kan ikke finde startkolonne (E eller D).")

    s_end = None
    if idx_F in in_cols:
        s_end = coerce(sub[idx_F])

    anns = []
    for ridx in sub.index:
        st = s_start.loc[ridx]
        if pd.isna(st):
            continue
        en = s_end.loc[ridx] if s_end is not None else pd.NaT
        anns.append({
            "start": pd.to_datetime(st).to_pydatetime(),
            "end": (pd.to_datetime(en).to_pydatetime() if not pd.isna(en) else None),
            "row": int(ridx) + 1,  # Excel-rækkenummer
        })
    if not anns:
        raise ValueError(f"Ingen gyldige tider i {xls_path} fra række {start_row}+.")
    return anns




def pick_channel(td: TdmsFile, preferred=("Recording", "ECG")):
    """
    Vælg kanal; prøv ("Recording","ECG") først (som vi lavede i trimming),
    ellers første kanal i første gruppe.
    """
    try:
        ch = td[preferred[0]][preferred[1]]
        return preferred[0], preferred[1], ch
    except Exception:
        for g in td.groups():
            chans = g.channels()
            if chans:
                return g.name, chans[0].name, chans[0]
    raise RuntimeError("Ingen kanaler fundet.")


def _inc_to_seconds(inc):
    """Robust konvertering af wf_increment til sekunder pr. sample."""
    if isinstance(inc, pd.Timedelta):
        return inc.total_seconds()
    try:
        import numpy as np
        if isinstance(inc, np.timedelta64):
            return pd.to_timedelta(inc).total_seconds()
    except Exception:
        pass
    # Fald tilbage: antag tal (allerede i sekunder)
    return float(inc)

def channel_time_bounds(ch, expect_fs=(100, 2000)):
    """
    Returnér (t0, t1, fs, n). Hvis fs er urimelig for EKG, prøv at korrigere
    wf_increment som minutter eller timer -> sekunder.
    """
    props = ch.properties
    inc_s = _inc_to_seconds(props["wf_increment"])  # s/sample
    fs = 1.0 / inc_s

    # Heuristisk enheds-fix: EKG bør ligge i ~100–2000 Hz (typisk 256/512)
    if not (expect_fs[0] <= fs <= expect_fs[1]):
        fs_try_min = 1.0 / (inc_s * 60.0)     # antag inc var i minutter
        fs_try_hour = 1.0 / (inc_s * 3600.0)  # antag inc var i timer
        if expect_fs[0] <= fs_try_min <= expect_fs[1]:
            inc_s *= 60.0
            fs = fs_try_min
            print("[INFO] Justerede wf_increment fra minutter -> sekunder.")
        elif expect_fs[0] <= fs_try_hour <= expect_fs[1]:
            inc_s *= 3600.0
            fs = fs_try_hour
            print("[INFO] Justerede wf_increment fra timer -> sekunder.")
        else:
            print(f"[ADVARSEL] Mærkelig fs={fs:.4f} Hz (ingen enhedskorrektion mulig).")

    t0 = pd.to_datetime(props["wf_start_time"]).to_pydatetime()
    n = len(ch)
    dur = timedelta(seconds=(n - 1) / fs) if n > 1 else timedelta(0)
    t1 = t0 + dur
    return t0, t1, fs, n

def align_annotation_to_file_window(st, en, t0, t1, minutes_before_start, minutes_after_end):
    """
    Brug st/en som de er, hvis segmentet (st-Δ, en+Δ) overlapper [t0,t1].
    Ellers prøv dag-shifts i rækkefølgen [0, -1, +1, -2, +2] og vælg bedste overlap
    (ved tie: vælg med mindst |shift|).
    Returnerer (st_adj, en_adj, overlap_seconds).
    """
    def seg(s, e):
        seg_start = s - timedelta(minutes=minutes_before_start)
        seg_end   = (e if e is not None else s) + timedelta(minutes=minutes_after_end)
        return seg_start, seg_end

    def overlap_sec(s, e):
        ss, ee = seg(s, e)
        return (min(ee, t1) - max(ss, t0)).total_seconds()

    # 0) hvis allerede overlap → behold
    if overlap_sec(st, en) > 0:
        return st, en, overlap_sec(st, en)

    # 1) prøv skift (prioritér 0, så -1/+1…)
    best = (st, en); best_ov = -1e9; best_shift = 0
    for shift in (0, -1, 1, -2, 2):
        s = st + timedelta(days=shift)
        e = en + timedelta(days=shift) if en is not None else None
        ov = overlap_sec(s, e)
        if ov > best_ov or (ov == best_ov and abs(shift) < abs(best_shift)):
            best, best_ov, best_shift = (s, e), ov, shift
    return best[0], best[1], best_ov

def extract_segment_by_datetime(ch, seg_start_dt, seg_end_dt, t0, fs, n):
    """
    Konverter (seg_start_dt, seg_end_dt) til sample-indeks og udtræk data.
    Klipper til filens grænser.
    Returnerer (sig, ts) hvor ts er numpy array i sekunder relativt til seg_start_dt.
    """
    # klip til [t0, t1]
    t1 = t0 + timedelta(seconds=(n - 1) / fs) if n > 1 else t0
    seg_start_dt = max(seg_start_dt, t0)
    seg_end_dt = min(seg_end_dt, t1)

    i0 = int(round((seg_start_dt - t0).total_seconds() * fs))
    i1 = int(round((seg_end_dt   - t0).total_seconds() * fs)) + 1  # inklusivt
    i0 = max(0, min(i0, n-1))
    i1 = max(i0+1, min(i1, n))

    sig = ch[i0:i1]
    # tidsakse relativ til segment-start (sek)
    ts = np.arange(i1 - i0) / fs
    return sig, ts, i0, i1


def plot_segment(ts, sig, ann_start_rel_sec, ann_end_rel_sec, out_png, title,
                 auto_ylim_percentiles=(1, 99)):
    """Tegn signalet så linjen ligger over markeringen og autoskaler y-aksen."""
    fig, ax = plt.subplots(figsize=(12, 4))
    # tegn markering først (så linjen kommer øverst)
    if ann_end_rel_sec is not None:
        ax.axvspan(ann_start_rel_sec, ann_end_rel_sec, alpha=0.15, color="0.7", zorder=1)
    ax.axvline(ann_start_rel_sec, linestyle="--", zorder=2)
    if ann_end_rel_sec is not None:
        ax.axvline(ann_end_rel_sec, linestyle="--", zorder=2)

    # selve signalet
    ax.plot(ts, sig, linewidth=0.8, zorder=3)

    # valgfrit: autoskalér y-aksen på percentiler for at undgå outliers der “flader” plottet
    if auto_ylim_percentiles is not None and len(sig) > 0:
        lo, hi = np.nanpercentile(sig, auto_ylim_percentiles)
        if np.isfinite(lo) and np.isfinite(hi) and hi > lo:
            pad = 0.05 * (hi - lo)
            ax.set_ylim(lo - pad, hi + pad)

    if len(ts) > 0:
        ax.set_xlim(ts[0], ts[-1])

    ax.set_xlabel("Tid [s] relativt til segment-start")
    ax.set_ylabel("EKG [unit]")
    ax.set_title(title)
    fig.tight_layout()
    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=150)
    plt.close(fig)


def _td_from_time_series(s: pd.Series) -> pd.Series:
    """Robust: 'HH:MM:SS' -> Timedelta; Excel-tal -> dage -> Timedelta."""
    if np.issubdtype(s.dtype, np.number):
        # Excel time as fraction of a day
        return pd.to_timedelta(s, unit="d", errors="coerce")
    # strings like '07:26:57', '7:26', etc.
    return pd.to_timedelta(s.astype(str).str.strip(), errors="coerce")

def _coerce_date(s: pd.Series) -> pd.Series:
    """Dato-kolonne til datetime.date (dayfirst=True)."""
    out = pd.to_datetime(s, errors="coerce", dayfirst=True)
    return out.dt.date

def _norm(txt):
    if not isinstance(txt, str):
        txt = str(txt)
    return "".join(ch for ch in txt.lower() if ch.isalnum())

# def load_annotations_from_excel_eeg_first(xls_path: str | Path, header_scan_rows: int = 20):
    """
    Læs en Excel som på dit screenshot og lav annoteringer pr. række:
      start = EEG start  (E), ellers Klinisk start (D)
      slut  = EEG slut   (G), ellers Klinisk slut  (F)
      dato  = 'Dato' kolonnen (C)
    Returnerer: [{"start": dt, "end": dt|None, "row": excel_row}, ...]
    """
    xls_path = Path(xls_path)
    engine = "xlrd" if xls_path.suffix.lower() == ".xls" else None
    df = pd.read_excel(xls_path, header=None, engine=engine)

    # 1) find header-rækken ved at lede efter kolonne-teksterne
    target_map = {
        "dato": None,
        "anfaldsstarteegttmmss": None,
        "anfaldsstop eegttmmss": None,
        "anfaldsstartkliniskttmmss": None,
        "anfaldsstopkliniskttmmss": None,
    }
    header_row = None
    for r in range(min(header_scan_rows, len(df))):
        row = df.iloc[r].tolist()
        for c, v in enumerate(row):
            key = _norm(v)
            if key in target_map and target_map[key] is None:
                target_map[key] = c
        # har vi mindst 'dato' og én startkolonne?
        if target_map["dato"] is not None and (
            target_map["anfaldsstarteegttmmss"] is not None
            or target_map["anfaldsstartkliniskttmmss"] is not None
        ):
            header_row = r
            break
    if header_row is None:
        raise ValueError("Kunne ikke finde header-rækken (’Dato’, ’Anfaldsstart EEG/Klinisk’).")

    c_date   = target_map["dato"]
    c_eeg_s  = target_map["anfaldsstarteegttmmss"]
    c_eeg_e  = target_map["anfaldsstop eegttmmss"]
    c_cli_s  = target_map["anfaldsstartkliniskttmmss"]
    c_cli_e  = target_map["anfaldsstopkliniskttmmss"]

    data = df.iloc[header_row+1:].copy()
    # 2) parse dato og tider
    date_col = _coerce_date(data[c_date])
    eeg_start_td = _td_from_time_series(data[c_eeg_s]) if c_eeg_s is not None else pd.Series([pd.NaT]*len(data), index=data.index)
    eeg_end_td   = _td_from_time_series(data[c_eeg_e]) if c_eeg_e is not None else pd.Series([pd.NaT]*len(data), index=data.index)
    cli_start_td = _td_from_time_series(data[c_cli_s]) if c_cli_s is not None else pd.Series([pd.NaT]*len(data), index=data.index)
    cli_end_td   = _td_from_time_series(data[c_cli_e]) if c_cli_e is not None else pd.Series([pd.NaT]*len(data), index=data.index)

    # 3) kombiner dato + tid (prioritér EEG)
    annotations = []
    for ridx in data.index:
        d = date_col.loc[ridx]
        if pd.isna(d):
            continue
        # byg datetimes
        st_eeg = pd.Timestamp.combine(d, pd.Timestamp(0) .time()) + (eeg_start_td.loc[ridx] if pd.notna(eeg_start_td.loc[ridx]) else pd.Timedelta(0))
        st_cli = pd.Timestamp.combine(d, pd.Timestamp(0) .time()) + (cli_start_td.loc[ridx] if pd.notna(cli_start_td.loc[ridx]) else pd.Timedelta(0))
        en_eeg = pd.Timestamp.combine(d, pd.Timestamp(0) .time()) + (eeg_end_td.loc[ridx]   if pd.notna(eeg_end_td.loc[ridx])   else pd.Timedelta(0))
        en_cli = pd.Timestamp.combine(d, pd.Timestamp(0) .time()) + (cli_end_td.loc[ridx]   if pd.notna(cli_end_td.loc[ridx])   else pd.Timedelta(0))

        # vælg start
        st = st_eeg if (c_eeg_s is not None and pd.notna(eeg_start_td.loc[ridx])) else (
             st_cli if (c_cli_s is not None and pd.notna(cli_start_td.loc[ridx])) else pd.NaT)
        if pd.isna(st):
            continue  # ingen start -> ignorer række

        # vælg slut
        en = None
        if c_eeg_e is not None and pd.notna(eeg_end_td.loc[ridx]):
            en = en_eeg
        elif c_cli_e is not None and pd.notna(cli_end_td.loc[ridx]):
            en = en_cli

        annotations.append({
            "start": pd.to_datetime(st).to_pydatetime(),
            "end": (pd.to_datetime(en).to_pydatetime() if en is not None else None),
            "row": int(ridx) + 1,  # rapportér som Excel-rækkenr (1-baseret)
        })

    if not annotations:
        raise ValueError("Ingen gyldige annoteringer (manglede dato/tider).")
    return annotations


def _norm(s):
    s = "" if s is None else str(s)
    return "".join(ch for ch in s.lower() if ch.isalnum())  # kun a-z0-9

def _td_from_time_series(s: pd.Series) -> pd.Series:
    if np.issubdtype(s.dtype, np.number):  # Excel-serial som fraktion af døgn
        return pd.to_timedelta(s, unit="d", errors="coerce")
    return pd.to_timedelta(s.astype(str).str.strip(), errors="coerce")

def _coerce_date(s: pd.Series) -> pd.Series:
    out = pd.to_datetime(s, errors="coerce", dayfirst=True)
    return out.dt.date

def load_annotations_from_excel_eeg_first(xls_path: str | Path, header_scan_rows: int = 20):
    """
    Finder kolonner via tokens:
      - Dato: indeholder 'dato'
      - EEG start: indeholder både 'anfaldsstart' og 'eeg'
      - EEG stop : indeholder både 'anfaldsstop'  og 'eeg'
      - Klinisk start/stop tilsvarende med 'klinisk'
    Returnerer [{"start": dt, "end": dt|None, "row": excel_row}, ...]
    """
    xls_path = Path(xls_path)
    engine = "xlrd" if xls_path.suffix.lower() == ".xls" else None
    df = pd.read_excel(xls_path, header=None, engine=engine)

    # --- find header-række + kolonneindekser via token-match ---
    c_date = c_eeg_s = c_eeg_e = c_cli_s = c_cli_e = None
    header_row = None

    def match_tokens(cell, must_have, must_not=None):
        key = _norm(cell)
        if not key:
            return False
        ok = all(tok in key for tok in must_have)
        if must_not:
            ok = ok and all(tok not in key for tok in must_not)
        return ok

    for r in range(min(header_scan_rows, len(df))):
        row = df.iloc[r]
        for c, v in row.items():
            if c_date is None and match_tokens(v, ["dato"]):
                c_date = c
            if c_eeg_s is None and match_tokens(v, ["anfaldsstart", "eeg"]):
                c_eeg_s = c
            if c_eeg_e is None and match_tokens(v, ["anfaldsstop", "eeg"]):
                c_eeg_e = c
            if c_cli_s is None and match_tokens(v, ["anfaldsstart", "klinisk"]):
                c_cli_s = c
            if c_cli_e is None and match_tokens(v, ["anfaldsstop", "klinisk"]):
                c_cli_e = c
        if c_date is not None and (c_eeg_s is not None or c_cli_s is not None):
            header_row = r
            break

    if header_row is None:
        raise ValueError("Kunne ikke finde header-rækken (Dato/EEG/Klinisk).")

    data = df.iloc[header_row+1:].copy()

    # --- parse dato + tider ---
    date_col = _coerce_date(data[c_date])

    def safe_td(col_idx):
        if col_idx is None or col_idx not in data.columns:
            return pd.Series([pd.NaT]*len(data), index=data.index)
        return _td_from_time_series(data[col_idx])

    eeg_s_td = safe_td(c_eeg_s)
    eeg_e_td = safe_td(c_eeg_e)
    cli_s_td = safe_td(c_cli_s)
    cli_e_td = safe_td(c_cli_e)

    anns = []
    for ridx in data.index:
        d = date_col.loc[ridx]
        if pd.isna(d):
            continue
        base = pd.Timestamp.combine(d, pd.Timestamp(0).time())

        st = base + (eeg_s_td.loc[ridx] if pd.notna(eeg_s_td.loc[ridx]) else cli_s_td.loc[ridx])
        if pd.isna(st):
            continue

        en = None
        if pd.notna(eeg_e_td.loc[ridx]):
            en = base + eeg_e_td.loc[ridx]
        elif pd.notna(cli_e_td.loc[ridx]):
            en = base + cli_e_td.loc[ridx]

        anns.append({
            "start": pd.to_datetime(st).to_pydatetime(),
            "end": (pd.to_datetime(en).to_pydatetime() if en is not None else None),
            "row": int(ridx) + 1,
        })

    if not anns:
        raise ValueError("Ingen gyldige annoteringer (manglende dato/tid).")
    return anns



# ------------------ Hoved-flow ------------------

def _sanitize_end_time(st, en, default_after_min=2):
    """
    Gør 'en' brugbar:
      - NaT/None -> st + default_after_min
      - Kun klokkeslæt (år <= 1901) -> brug datoen fra st (og +1 dag hvis den krydser midnat)
      - Hvis afstanden er urealistisk stor (> 48h) -> fald tilbage til st + default_after_min
    """
    if en is None or pd.isna(en):
        return st + timedelta(minutes=default_after_min)

    # Excel "time only" (år 1899/1900/1901)
    if getattr(en, "year", 3000) <= 1901:
        en = en.replace(year=st.year, month=st.month, day=st.day)
        if en < st:
            en = en + timedelta(days=1)

    # Outliers (fx 2025 mod 2016)
    if abs((en - st).total_seconds()) > 48 * 3600:
        return st + timedelta(minutes=default_after_min)

    return en

def qa_extract_and_plot(
    trimmed_root: str | Path,
    patients_root_with_excels: str | Path,
    minutes_before_start: int = 5,
    minutes_after_end: int = 5,
    excel_glob: str = "*.xls*",   # .xls og .xlsx
):
    trimmed_root = Path(trimmed_root)
    excel_root = Path(patients_root_with_excels)

    # Tillad direkte sti til én Excel
    if excel_root.is_file() and excel_root.suffix.lower().startswith(".xls"):
        excel_files = [excel_root]
        excel_root = excel_root.parent
    else:
        excel_files = list(excel_root.rglob(excel_glob))

    if not excel_files:
        print(f"Ingen Excel-filer fundet under: {excel_root}")
        return

    summary_rows = []

    for xls in excel_files:
        pid = extract_patient_id_from_path(xls)
        if pid is None:
            print(f"[SKIP] Kan ikke udlede patient-ID fra: {xls}")
            continue

        # --- Find trimmede TDMS: 1) spejlet mappe, 2) patient-søgning, 3) alt ---
        try:
            rel_dir = xls.parent.relative_to(excel_root)
            cand_dir = trimmed_root / rel_dir
        except Exception:
            cand_dir = trimmed_root

        tdms_list = sorted(cand_dir.glob("*_trimmed.tdms"))
        if not tdms_list:
            tdms_list = sorted(trimmed_root.rglob(f"**/Patient*{pid}*/*_trimmed.tdms")) + \
                        sorted(trimmed_root.rglob(f"**/patient*{pid}*/*_trimmed.tdms"))
        if not tdms_list:
            tdms_list = sorted(trimmed_root.rglob("**/*_trimmed.tdms"))
        if not tdms_list:
            print(f"[ADVARSEL] Ingen *_trimmed.tdms fundet for Patient {pid} i {trimmed_root}")
            continue

        # -- Læs annoteringer
        try:
            anns = load_annotations_from_excel_eeg_first(xls, header_scan_rows=20)


        except Exception as e:
            print(f"[FEJL] Kunne ikke læse annoteringer fra {xls}: {e}")
            continue
        if not anns:
            print(f"[INFO] Ingen gyldige annoteringer i {xls}")
            continue

        print(f"\n== Patient {pid}: {xls.name} | {len(anns)} annoteringer ==")


        # -- For hver annotering
        for j, a in enumerate(anns, 1):
            st = a["start"]
            en = _sanitize_end_time(st, a["end"], default_after_min=2)

            seg_start = st - timedelta(minutes=minutes_before_start)
            seg_end   = en + timedelta(minutes=minutes_after_end)

            # Definér på forhånd (så de findes, selv hvis vi rammer except)
            ann_start_rel = (st - seg_start).total_seconds()
            ann_end_rel = (en - seg_start).total_seconds() if en else None

            matched = False
            first_bounds_msg = None

            print(f"  ann#{j}: st={st}, en={en}")
            seg_start = st - timedelta(minutes=minutes_before_start)
            seg_end   = (en if en else st) + timedelta(minutes=minutes_after_end)
            print(f"           seg_start={seg_start}, seg_end={seg_end}")

            for tdms_path in tdms_list:
                try:
                    with TdmsFile.open(tdms_path) as td:
                        _, _, ch = pick_channel(td)
                        t0, t1, fs, n = channel_time_bounds(ch)

                        # justér annotering til filvinduet
                        st_adj, en_adj, overlap = align_annotation_to_file_window(
                            st, en, t0, t1, minutes_before_start, minutes_after_end
                        )
                        print(f"           {tdms_path.name}: aligned st={st_adj}, en={en_adj}, overlap={overlap:.1f}s")

                        if overlap <= 0:
                            continue  # ingen brugbar overlap for denne fil, prøv næste

                        seg_start = st_adj - timedelta(minutes=minutes_before_start)
                        seg_end   = (en_adj if en_adj is not None else st_adj) + timedelta(minutes=minutes_after_end)

                        # udtræk segment
                        sig, ts, i0, i1 = extract_segment_by_datetime(ch, seg_start, seg_end, t0, fs, n)
                        ann_start_rel = max(0.0, (st_adj - seg_start).total_seconds())
                        ann_end_rel = (en_adj - seg_start).total_seconds() if en_adj is not None else None
                        if ann_end_rel is not None:
                            ann_end_rel = max(0.0, ann_end_rel)

                        # Debug-print
                        print(f"  ✓ ann#{j}: fs={fs:.2f}Hz, samples={len(sig)}, i0={i0}, i1={i1}, "
                              f"seg_dur={ts[-1]-ts[0] if len(ts) else 0:.2f}s")

                        # Plot
                        qa_dir = tdms_path.parent / "QA_plots"
                        title = f"Patient {pid} | {tdms_path.stem} | ann#{j} (row {a['row']})"
                        out_png = qa_dir / f"{tdms_path.stem}_ann{j:03d}.png"
                        plot_segment(ts, sig, ann_start_rel, ann_end_rel, out_png, title)

                        # Summary
                        summary_rows.append({
                            "patient_id": pid,
                            "excel": str(xls),
                            "tdms": str(tdms_path),
                            "ann_index": j,
                            "row_in_excel": a["row"],
                            "fs_Hz": fs,
                            "ch_n": n,
                            "file_t0": pd.to_datetime(t0),
                            "file_t1": pd.to_datetime(t1),
                            "ann_start": pd.to_datetime(st),
                            "ann_end": pd.to_datetime(en) if en else pd.NaT,
                            "seg_start": pd.to_datetime(seg_start),
                            "seg_end": pd.to_datetime(seg_end),
                            "i0": i0,
                            "i1": i1,
                            "png": str(out_png),
                            "status": "ok",
                        })
                        matched = True
                        break

                except Exception as e:
                    # Log fejlen uden at bruge udefinerede variabler
                    print(f"  [FEJL] {tdms_path}: {type(e).__name__}: {e}")
                    summary_rows.append({
                        "patient_id": pid, "excel": str(xls), "tdms": str(tdms_path),
                        "ann_index": j, "row_in_excel": a["row"], "status": f"error: {type(e).__name__}: {e}"
                    })

            if not matched:
                msg = first_bounds_msg or "ingen TDMS åbnet"
                print(f"  [ADVARSEL] ann#{j}: fandt ingen TDMS med overlap af {seg_start}–{seg_end} ({msg})")
                summary_rows.append({
                    "patient_id": pid, "excel": str(xls), "tdms": "",
                    "ann_index": j, "row_in_excel": a["row"], "status": "no_matching_tdms",
                    "ann_start": pd.to_datetime(st),
                    "ann_end": pd.to_datetime(en) if en else pd.NaT,
                    "seg_start": pd.to_datetime(seg_start),
                    "seg_end": pd.to_datetime(seg_end),
                })


# ------------------ Kørsel ------------------

if __name__ == "__main__":
    # Justér disse to stier:
    TRIMMED_ROOT = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\recording 1"
    PATIENTS_ROOT_WITH_EXCELS = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\Patient 5.xls"

    qa_extract_and_plot(
        trimmed_root=TRIMMED_ROOT,
        patients_root_with_excels=PATIENTS_ROOT_WITH_EXCELS,
        minutes_before_start=1,
        minutes_after_end=1,
        excel_glob="*.xls*",  # .xls og .xlsx
    )


[FEJL] Kunne ikke læse annoteringer fra E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\Patient 5.xls: Ingen gyldige annoteringer (manglende dato/tid).


In [20]:
# --- QA: Excel (EEG-first) -> segmenter fra trimmede TDMS og plots ---
# Kør alt i én celle. Tilpas stier i "MAIN" nederst.

from pathlib import Path
from datetime import timedelta, datetime
import os, re, traceback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nptdms import TdmsFile


# =========================
# Utilities
# =========================

def _col_letter(idx: int) -> str:
    # 0->A, 1->B ...
    return chr(ord('A') + idx)

def _norm(s):
    s = "" if s is None else str(s)
    return "".join(ch for ch in s.lower() if ch.isalnum())  # a-z0-9

def extract_patient_id_from_path(p: Path) -> int | None:
    parts = list(p.parts) + [p.stem]
    for part in parts:
        m = re.search(r'patient[ _-]?(\d+)', part, flags=re.IGNORECASE)
        if m:
            try:
                return int(m.group(1))
            except ValueError:
                pass
    return None


# =========================
# Excel: EEG-first loader
# =========================

def _td_from_time_series(s: pd.Series) -> pd.Series:
    """Robust: Excel-tal => fraktion af døgn; tekst 'HH:MM:SS' => Timedelta."""
    if np.issubdtype(s.dtype, np.number):
        return pd.to_timedelta(s, unit="d", errors="coerce")
    return pd.to_timedelta(s.astype(str).str.strip(), errors="coerce")

def _coerce_date(s: pd.Series) -> pd.Series:
    """Dato til datetime.date (dayfirst=True). Håndterer også Excel-serial via to_datetime."""
    out = pd.to_datetime(s, errors="coerce", dayfirst=True)
    # Nogle xls har dato som tal -> bliver også korrekt her
    return out.dt.date

def _find_header_and_columns(df: pd.DataFrame, header_scan_rows: int = 30):
    """
    Find header-rækken ved at lede efter 'Dato' + mindst én start-kolonne
    i *samme* række. Returnér kolonneindekser fra den række.
    Fallback til (række 7, C/E/G/D/F) hvis ikke fundet.
    """
    def match_tokens(cell, must_have, must_not=None):
        key = _norm(cell)
        if not key:
            return False
        ok = all(tok in key for tok in must_have)
        if must_not:
            ok = ok and all(tok not in key for tok in must_not)
        return ok

    scan_limit = min(header_scan_rows, len(df))
    for r in range(scan_limit):
        row = df.iloc[r]

        # Nulstil for hver række (VIGTIGT)
        c_date = c_eeg_s = c_eeg_e = c_cli_s = c_cli_e = None

        for c, v in row.items():
            if c_date is None and match_tokens(v, ["dato"]):
                c_date = c
            if c_eeg_s is None and match_tokens(v, ["anfaldsstart", "eeg"]):
                c_eeg_s = c
            if c_eeg_e is None and match_tokens(v, ["anfaldsstop", "eeg"]):
                c_eeg_e = c
            if c_cli_s is None and match_tokens(v, ["anfaldsstart", "klinisk"]):
                c_cli_s = c
            if c_cli_e is None and match_tokens(v, ["anfaldsstop", "klinisk"]):
                c_cli_e = c

        # Krav: Dato + (EEG_start eller Klinisk_start) fundet i samme række
        if c_date is not None and (c_eeg_s is not None or c_cli_s is not None):
            return r, c_date, c_eeg_s, c_eeg_e, c_cli_s, c_cli_e

    # ---- Fallback til dit layout (række 7; C/E/G/D/F) ----
    header_row = 6  # 0-indekseret -> Excel-række 7
    cols = set(df.columns)
    def ci(letter):
        idx = ord(letter.upper()) - ord('A')
        return idx if idx in cols else None

    c_date  = ci('C')
    c_eeg_s = ci('E')
    c_eeg_e = ci('G')
    c_cli_s = ci('D')
    c_cli_e = ci('F')
    return header_row, c_date, c_eeg_s, c_eeg_e, c_cli_s, c_cli_e


def load_annotations_from_excel_eeg_first(xls_path: str | Path, header_scan_rows: int = 30):
    """
    Læs annoteringer som på dit ark:
      start = EEG start  (prioritet), ellers Klinisk start
      slut  = EEG stop   (prioritet), ellers Klinisk stop
      dato  = 'Dato'
    Returnerer liste af dicts: [{"start": dt, "end": dt|None, "row": excel_row}, ...]
    """
    xls_path = Path(xls_path)
    engine = "xlrd" if xls_path.suffix.lower() == ".xls" else None
    df = pd.read_excel(xls_path, header=None, engine=engine)

    header_row, c_date, c_eeg_s, c_eeg_e, c_cli_s, c_cli_e = _find_header_and_columns(df, header_scan_rows)
    data = df.iloc[header_row+1:].copy()

    # Debug (valgfrit): print valgte kolonner
    chosen = {
        "header_row(1-based)": header_row + 1,
        "Dato": _col_letter(c_date) if c_date is not None else None,
        "EEG_start": _col_letter(c_eeg_s) if c_eeg_s is not None else None,
        "EEG_stop": _col_letter(c_eeg_e) if c_eeg_e is not None else None,
        "Klinisk_start": _col_letter(c_cli_s) if c_cli_s is not None else None,
        "Klinisk_stop": _col_letter(c_cli_e) if c_cli_e is not None else None,
    }
    print("Excel-kolonner:", chosen)

    # Parse dato + tider
    if c_date is None or c_date not in data.columns:
        raise ValueError("Fandt ikke 'Dato'-kolonnen. Justér header/Fallback.")

    date_col = _coerce_date(data[c_date])

    def safe_td(col_idx):
        if col_idx is None or col_idx not in data.columns:
            return pd.Series([pd.NaT]*len(data), index=data.index)
        return _td_from_time_series(data[col_idx])

    eeg_s_td = safe_td(c_eeg_s)
    eeg_e_td = safe_td(c_eeg_e)
    cli_s_td = safe_td(c_cli_s)
    cli_e_td = safe_td(c_cli_e)

    anns = []
    for ridx in data.index:
        d = date_col.loc[ridx]
        if pd.isna(d):
            continue
        base = pd.Timestamp.combine(d, pd.Timestamp(0).time())
        # vælg start
        st_td = eeg_s_td.loc[ridx] if pd.notna(eeg_s_td.loc[ridx]) else cli_s_td.loc[ridx]
        if pd.isna(st_td):
            continue
        st = base + st_td

        # vælg slut
        en = None
        if pd.notna(eeg_e_td.loc[ridx]):
            en = base + eeg_e_td.loc[ridx]
        elif pd.notna(cli_e_td.loc[ridx]):
            en = base + cli_e_td.loc[ridx]

        # kryds af midnat (hvis slut < start)
        if en is not None and en < st:
            en = en + timedelta(days=1)

        anns.append({
            "start": pd.to_datetime(st).to_pydatetime(),
            "end": (pd.to_datetime(en).to_pydatetime() if en is not None else None),
            "row": int(ridx) + 1,  # Excel-rækkenummer
        })

    if not anns:
        raise ValueError("Ingen gyldige annoteringer (manglende dato/tid).")
    return anns


# =========================
# TDMS helpers
# =========================

def pick_channel(td: TdmsFile, preferred=("Recording", "ECG")):
    try:
        ch = td[preferred[0]][preferred[1]]
        return preferred[0], preferred[1], ch
    except Exception:
        for g in td.groups():
            chans = g.channels()
            if chans:
                return g.name, chans[0].name, chans[0]
    raise RuntimeError("Ingen kanaler fundet.")

def _inc_to_seconds(inc):
    if isinstance(inc, pd.Timedelta):
        return inc.total_seconds()
    try:
        if isinstance(inc, np.timedelta64):
            return pd.to_timedelta(inc).total_seconds()
    except Exception:
        pass
    return float(inc)

def channel_time_bounds(ch, expect_fs=(50, 4000)):
    """Returnér (t0, t1, fs, n). Heuristik retter wf_increment hvis enhed er forkert."""
    props = ch.properties
    inc_s = _inc_to_seconds(props["wf_increment"])  # s/sample
    fs = 1.0 / inc_s

    if not (expect_fs[0] <= fs <= expect_fs[1]):
        fs_try_min = 1.0 / (inc_s * 60.0)
        fs_try_hour = 1.0 / (inc_s * 3600.0)
        if expect_fs[0] <= fs_try_min <= expect_fs[1]:
            inc_s *= 60.0; fs = fs_try_min
            print("[INFO] Justerede wf_increment fra minutter -> sekunder.")
        elif expect_fs[0] <= fs_try_hour <= expect_fs[1]:
            inc_s *= 3600.0; fs = fs_try_hour
            print("[INFO] Justerede wf_increment fra timer -> sekunder.")
        else:
            print(f"[ADVARSEL] Uventet fs={fs:.4f} Hz – tjek wf_increment.")

    t0 = pd.to_datetime(props["wf_start_time"]).to_pydatetime()
    n  = len(ch)
    dur = timedelta(seconds=(n - 1) / fs) if n > 1 else timedelta(0)
    t1 = t0 + dur
    return t0, t1, fs, n

def extract_segment_by_datetime(ch, seg_start_dt, seg_end_dt, t0, fs, n):
    # klip til filens vindue
    t1 = t0 + timedelta(seconds=(n - 1) / fs) if n > 1 else t0
    seg_start_dt = max(seg_start_dt, t0)
    seg_end_dt   = min(seg_end_dt, t1)

    i0 = int(round((seg_start_dt - t0).total_seconds() * fs))
    i1 = int(round((seg_end_dt   - t0).total_seconds() * fs)) + 1
    i0 = max(0, min(i0, n-1))
    i1 = max(i0+1, min(i1, n))

    sig = ch[i0:i1]
    ts = np.arange(i1 - i0) / fs  # sek fra segment-start
    return sig, ts, i0, i1

def plot_segment(ts, sig, ann_start_rel_sec, ann_end_rel_sec, out_png, title,
                 auto_ylim_percentiles=(1, 99)):
    fig, ax = plt.subplots(figsize=(12, 4))
    if ann_end_rel_sec is not None:
        ax.axvspan(ann_start_rel_sec, ann_end_rel_sec, alpha=0.15, color="0.7", zorder=1)
    ax.axvline(ann_start_rel_sec, linestyle="--", zorder=2)
    if ann_end_rel_sec is not None:
        ax.axvline(ann_end_rel_sec, linestyle="--", zorder=2)
    ax.plot(ts, sig, linewidth=0.8, zorder=3)

    if auto_ylim_percentiles is not None and len(sig) > 0:
        lo, hi = np.nanpercentile(sig, auto_ylim_percentiles)
        if np.isfinite(lo) and np.isfinite(hi) and hi > lo:
            pad = 0.05 * (hi - lo)
            ax.set_ylim(lo - pad, hi + pad)

    if len(ts) > 0:
        ax.set_xlim(ts[0], ts[-1])
    ax.set_xlabel("Tid [s] relativt til segment-start")
    ax.set_ylabel("EKG [unit]")
    ax.set_title(title)
    fig.tight_layout()
    out_png.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(out_png, dpi=150)
    plt.close(fig)


# =========================
# Core QA function
# =========================

def qa_extract_and_plot(
    trimmed_root: str | Path,
    excel_input: str | Path,
    minutes_before_start: int = 5,
    minutes_after_end: int = 5,
    excel_glob: str = "*.xls*",
):
    """
    trimmed_root: mappe (eller fil) med de trimmede TDMS (søger rekursivt efter *_trimmed.tdms)
    excel_input : mappe (eller fil) med Excel-annoteringer. Hvis fil, bruges kun den.
    """
    trimmed_root = Path(trimmed_root)
    excel_input  = Path(excel_input)

    # Saml TDMS-kandidater
    if trimmed_root.is_file() and trimmed_root.suffix.lower() == ".tdms":
        tdms_all = [trimmed_root]
        trimmed_root = trimmed_root.parent
    else:
        tdms_all = sorted(trimmed_root.rglob("**/*_trimmed.tdms"))
    if not tdms_all:
        print(f"[ADVARSEL] Ingen *_trimmed.tdms fundet under: {trimmed_root}")
        return

    # Saml Excel-filer
    if excel_input.is_file() and excel_input.suffix.lower().startswith(".xls"):
        excel_files = [excel_input]
        excel_root = excel_input.parent
    else:
        excel_root = excel_input
        excel_files = list(excel_root.rglob(excel_glob))
    if not excel_files:
        print(f"[ADVARSEL] Ingen Excel-filer fundet under: {excel_root}")
        return

    summary_rows = []
    print(f"Fandt {len(tdms_all)} TDMS og {len(excel_files)} Excel-filer.")

    for xls in excel_files:
        pid = extract_patient_id_from_path(xls)
        if pid is None:
            print(f"[SKIP] Kan ikke udlede patient-ID fra: {xls}")
            continue

        # Find TDMS for samme patient (først i samme relative mappe, derefter bredt)
        try:
            rel_dir = xls.parent.relative_to(excel_root)
            cand_dir = trimmed_root / rel_dir
        except Exception:
            cand_dir = trimmed_root
        tdms_list = sorted(cand_dir.glob("*_trimmed.tdms"))
        if not tdms_list:
            pat = f"*{pid}*"
            tdms_list = sorted(trimmed_root.rglob(f"**/Patient{pat}/*_trimmed.tdms")) + \
                        sorted(trimmed_root.rglob(f"**/patient{pat}/*_trimmed.tdms"))
        if not tdms_list:
            tdms_list = [t for t in tdms_all if str(pid) in t.as_posix()]

        # Læs annoteringer (EEG-first)
        try:
            anns = load_annotations_from_excel_eeg_first(xls, header_scan_rows=30)
        except Exception as e:
            print(f"[FEJL] Kunne ikke læse annoteringer fra {xls}: {e}")
            continue
        if not anns:
            print(f"[INFO] Ingen gyldige annoteringer i {xls}")
            continue

        print(f"\n== Patient {pid}: {xls.name} | {len(anns)} annoteringer ==")

        for j, a in enumerate(anns, 1):
            st, en = a["start"], a["end"]
            # Hvis slut mangler, brug 2 min default
            if en is None:
                en = st + timedelta(minutes=2)

            seg_start = st - timedelta(minutes=minutes_before_start)
            seg_end   = en + timedelta(minutes=minutes_after_end)
            print(f"  ann#{j}: st={st}, en={en}  -> seg={seg_start}..{seg_end}")

            matched = False
            first_bounds_msg = None

            for tdms_path in tdms_list:
                try:
                    with TdmsFile.open(tdms_path) as td:
                        _, _, ch = pick_channel(td)
                        t0, t1, fs, n = channel_time_bounds(ch)

                        if first_bounds_msg is None:
                            first_bounds_msg = f"{t0}..{t1} @ {fs:.2f}Hz"

                        # overlap?
                        if seg_end < t0 or seg_start > t1:
                            continue

                        # udtræk segment
                        sig, ts, i0, i1 = extract_segment_by_datetime(ch, seg_start, seg_end, t0, fs, n)
                        ann_start_rel = max(0.0, (st - seg_start).total_seconds())
                        ann_end_rel = max(0.0, (en - seg_start).total_seconds()) if en else None

                        seg_dur = (ts[-1] - ts[0]) if len(ts) else 0
                        print(f"    ✓ {tdms_path.name}: fs={fs:.2f}Hz, samples={len(sig)}, i0={i0}, i1={i1}, dur={seg_dur:.2f}s")

                        qa_dir = tdms_path.parent / "QA_plots"
                        title = f"Patient {pid} | {tdms_path.stem} | ann#{j} (row {a['row']})"
                        out_png = qa_dir / f"{tdms_path.stem}_ann{j:03d}.png"
                        plot_segment(ts, sig, ann_start_rel, ann_end_rel, out_png, title)

                        summary_rows.append({
                            "patient_id": pid, "excel": str(xls), "tdms": str(tdms_path),
                            "ann_index": j, "row_in_excel": a["row"], "fs_Hz": fs, "ch_n": n,
                            "file_t0": pd.to_datetime(t0), "file_t1": pd.to_datetime(t1),
                            "ann_start": pd.to_datetime(st), "ann_end": pd.to_datetime(en),
                            "seg_start": pd.to_datetime(seg_start), "seg_end": pd.to_datetime(seg_end),
                            "i0": i0, "i1": i1, "png": str(out_png), "status": "ok",
                        })
                        matched = True
                        break

                except Exception as e:
                    print(f"    [FEJL] {tdms_path.name}: {type(e).__name__}: {e}")
                    summary_rows.append({
                        "patient_id": pid, "excel": str(xls), "tdms": str(tdms_path),
                        "ann_index": j, "row_in_excel": a["row"],
                        "status": f"error: {type(e).__name__}: {e}"
                    })

            if not matched:
                print(f"    [ADVARSEL] ann#{j}: ingen TDMS-overlap (filvindue {first_bounds_msg})")
                summary_rows.append({
                    "patient_id": pid, "excel": str(xls), "tdms": "",
                    "ann_index": j, "row_in_excel": a["row"], "status": "no_matching_tdms",
                    "ann_start": pd.to_datetime(st), "ann_end": pd.to_datetime(en),
                    "seg_start": pd.to_datetime(seg_start), "seg_end": pd.to_datetime(seg_end),
                })

    # Gem summary i roden af trimmed_root
    if summary_rows:
        out_csv = Path(trimmed_root) / "QA_segments_summary.csv"
        pd.DataFrame(summary_rows).to_csv(out_csv, index=False)
        print(f"\nSummary gemt: {out_csv}")
    else:
        print("\nIngen plots/summary at gemme.")


# =========================
# MAIN – Tilpas disse to
# =========================

# Eksempel (peg til en patientmappe eller roden med alle trimmede outputs)
TRIMMED_ROOT = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\recording 1"
# Peg til én Excel-fil (eller en rodmappe med mange Excel-filer)
EXCEL_INPUT  = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\Patient 5.xls"

qa_extract_and_plot(
    trimmed_root=TRIMMED_ROOT,
    excel_input=EXCEL_INPUT,
    minutes_before_start=1,   # 1 min før start
    minutes_after_end=1,      # 1 min efter slut
    excel_glob="*.xls*",      # relevant hvis EXCEL_INPUT er en mappe
)


Fandt 1 TDMS og 1 Excel-filer.
Excel-kolonner: {'header_row(1-based)': 7, 'Dato': 'C', 'EEG_start': 'E', 'EEG_stop': None, 'Klinisk_start': 'D', 'Klinisk_stop': None}

== Patient 5: Patient 5.xls | 3 annoteringer ==
  ann#1: st=2016-10-13 07:22:37, en=2016-10-13 07:24:37  -> seg=2016-10-13 07:21:37..2016-10-13 07:25:37
    ✓ Patient 5_1_trimmed.tdms: fs=512.00Hz, samples=122881, i0=40445440, i1=40568321, dur=240.00s


C:\Users\kvjkv\AppData\Local\Temp\ipykernel_13168\2832436294.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  out = pd.to_datetime(s, errors="coerce", dayfirst=True)


  ann#2: st=2016-10-13 14:47:29, en=2016-10-13 14:49:29  -> seg=2016-10-13 14:46:29..2016-10-13 14:50:29
    ✓ Patient 5_1_trimmed.tdms: fs=512.00Hz, samples=122881, i0=54111744, i1=54234625, dur=240.00s
  ann#3: st=2016-10-13 15:36:30, en=2016-10-13 15:38:30  -> seg=2016-10-13 15:35:30..2016-10-13 15:39:30
    ✓ Patient 5_1_trimmed.tdms: fs=512.00Hz, samples=122881, i0=55617536, i1=55740417, dur=240.00s

Summary gemt: E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch data from Aarhus - Corrected\ePatch data\Patient 5\recording 1\QA_segments_summary.csv
